In [2]:
# !pip install chromadb sentence_transformers

In [3]:
import chromadb
from sentence_transformers import SentenceTransformer
import torch
from collections import defaultdict
import math

# ---------------------------------------------------------
# 1. 설정 (GPU 확인 등)
# ---------------------------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ---------------------------------------------------------
# 2. 모델 로드 (저장할 때 썼던 그 모델!)
# ---------------------------------------------------------
print("모델 로드 중...")
model = SentenceTransformer("dragonkue/BGE-m3-ko").to(device)

# ---------------------------------------------------------
# 3. ChromaDB 연결 (다운로드 받은 폴더 경로 지정)
# ---------------------------------------------------------
# path="./chroma_db" 는 압축 푼 폴더 이름과 같아야 합니다.
client = chromadb.PersistentClient(path="./chroma_db")

# 컬렉션 가져오기 (create가 아니라 get_collection 사용)
collection = client.get_collection(name="patent_claims")

print(f"✅ 데이터베이스 로드 완료! 총 데이터 수: {collection.count()}개")



c:\Users\kouls\anaconda3\envs\llm_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
모델 로드 중...
✅ 데이터베이스 로드 완료! 총 데이터 수: 589049개


### 특허 단위 점수 집계(Late Fusion Aggregated Scoring)
- 출원번호 하나당 모든 ‘매칭된 청구항’들의 유사도 점수를 통계적으로 합성해서 특허 단위 점수를 만듦
- 단점: 독립항이 핵심인데 종속항이 우연히 많이 매칭되면 점수가 잘못 올라갈 수 있음

In [4]:
# 200개 초과 검색되었을 때 re-ranking (z 정규화 수정ver)
import numpy as np

def multi_query_rerank(
    collection,
    model,
    query_list,
    per_query_top_k=200,
    final_top_k=200
):
    #------------------------------------------
    # 1) Q개의 query 문장 embedding
    #------------------------------------------
    query_embs = model.encode(query_list).tolist()


    #------------------------------------------
    # 2-1) query_list가 한 개일 경우, 검색 후 바로 return
    #------------------------------------------
    if len(query_list) == 1:
        return collection.query(query_embeddings=query_embs, n_results=per_query_top_k)


    #------------------------------------------
    # 2-2) query별 검색
    #------------------------------------------
    candidates = []  # 전체 후보 저장
    for emb in query_embs:
        r = collection.query(
            query_embeddings=[emb],
            n_results=per_query_top_k
        )
    
        distances = np.array(r["distances"][0])
        mean = distances.mean()
        std = distances.std() + 1e-9

        z_scores = (distances - mean) / std

        ids = r["ids"][0]
        docs = r["documents"][0]
        distances = r["distances"][0]
        metas = r["metadatas"][0]

        # 후보를 통합 리스트에 추가
        for pid, doc, meta, z, dist in zip(ids, docs, metas, z_scores, distances):
            candidates.append({
                "id": pid,
                "document": doc,
                'metadatas':meta,
                'distance':dist,
                'z-score':z
            })

    #------------------------------------------
    # 3) z-score 기준 오름차순 정렬 후 상위 final_top_k만 선택
    #------------------------------------------
    top_candidates = sorted(candidates, key=lambda x: x["z-score"])[:final_top_k]


    #------------------------------------------
    # 4) collection.query() 형식으로 재구성
    #------------------------------------------
    final_ids = [c["id"] for c in top_candidates]
    final_docs = [c["document"] for c in top_candidates]
    final_distances = [c["distance"] for c in top_candidates]
    final_metas = [c['metadatas'] for c in top_candidates]

    final_results = {
        "ids": [final_ids],
        "documents": [final_docs],
        "distances": [final_distances],
        "metadatas": [final_metas]
    }

    return final_results

### 기존 알고리즘

In [5]:
def many_claim_dis(results, TOP_K):
    # ----------------------------------------
    # 0. Chroma 결과 파싱
    # ----------------------------------------
    ids        = results["ids"][0]
    docs       = results["documents"][0]
    metas      = results["metadatas"][0]
    distances  = results["distances"][0]

    parsed = []
    for i in range(len(ids)):
        parsed.append({
            "id": ids[i],
            "document": docs[i],
            "metadata": metas[i],
            "distance": distances[i]
        })

    # ----------------------------------------
    # 1. 출원번호 기준 그룹화
    # ----------------------------------------
    grouped = defaultdict(list)
    for r in parsed:
        app_no = r["metadata"]["patent_id"]
        grouped[app_no].append(r)

    # ----------------------------------------
    # 2. 특허 단위 점수 계산
    #    방법: claim similarity들의 평균 + 대표 claim 보정
    # ----------------------------------------

    def similarity(d):
        return 1-d


    def compute_patent_score(claims):
        sims = [similarity(c["distance"]) for c in claims]   # 새로운 sim

        sims_sorted = sorted(sims, reverse=True)
        top3 = sims_sorted[:3]
        top3_avg = sum(top3) / len(top3)
        max_sim = sims_sorted[0]

        claim_count = len(claims)
        count_bonus = min(1.0, claim_count / 10.0)

        final_score =  top3_avg * 0.6 + max_sim * 0.3 + count_bonus * 0.1
    
        return final_score

    # ----------------------------------------
    # 3. 특허 단위 재랭킹
    # ----------------------------------------
    aggregated = []
    for app_no, claims in grouped.items():
        score = compute_patent_score(claims)

        # 대표 claim은 거리(distance)가 가장 낮은 claim 선택
        rep_claim = sorted(claims, key=lambda x: x["distance"])[0]
         # claims에서 id와 metadata를 제외하고 document와 distance만 저장
        
        filtered_claims = [
            {
                "id": c["id"],
                "document": c["document"],
                "distance": c["distance"]
            }
            for c in claims
        ]

        aggregated.append({
            "patent_id": app_no,
            "score": score,
            "top_claim": rep_claim["document"],
            "top_claim_no": rep_claim["metadata"]["claim_no"],
            "claims_found": len(claims),
            "claims": filtered_claims
        })

    # 점수 높은 순으로 재랭킹
    aggregated = sorted(aggregated, key=lambda x: x["score"], reverse=True)

    final_response = aggregated[:TOP_K]
    return final_response

### 하이브리드 서치 추가

In [6]:
# 1. 함수 import
from hybrid_search_function import hybrid_search

# 2. 쿼리 정의
query = [
    "영상 처리 3D Mapping 시스템 소프트웨어 데이터 처리 MCU 제어부", 
    "컴퓨터 입력 장치 키보드 터치판 사용자 인터페이스 소프트웨어 시스템"
]

# 3. multi_query_rerank 실행
results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

# 4. hybrid_search 실행 (BM25 추가)
final_response = hybrid_search(
    multi_query_results=results,
    query_list=query,
    top_k=30,
    vector_weight=0.7,
    bm25_weight=0.3
)

# 5. 결과 확인
for r in final_response:
    print(f"특허번호: {r['patent_id']}, 점수: {r['score']:.4f}, 청구항 수: {r['claims_found']}")

특허번호: 1020257003112, 점수: 4.4031, 청구항 수: 1
특허번호: 1020247038970, 점수: 3.8303, 청구항 수: 1
특허번호: 1020230170849, 점수: 3.7671, 청구항 수: 2
특허번호: 1020230169684, 점수: 3.7422, 청구항 수: 2
특허번호: 1020247027139, 점수: 3.3084, 청구항 수: 1
특허번호: 1020247009674, 점수: 2.8606, 청구항 수: 1
특허번호: 1020230066788, 점수: 2.7369, 청구항 수: 1
특허번호: 1020210102580, 점수: 2.6509, 청구항 수: 1
특허번호: 1020247026176, 점수: 2.6119, 청구항 수: 1
특허번호: 1020190081524, 점수: 2.5680, 청구항 수: 1
특허번호: 1020220116661, 점수: 2.3360, 청구항 수: 1
특허번호: 1020227009089, 점수: 2.2658, 청구항 수: 1
특허번호: 1020247014160, 점수: 2.2551, 청구항 수: 2
특허번호: 1020230166856, 점수: 2.2526, 청구항 수: 6
특허번호: 1020220039872, 점수: 2.2064, 청구항 수: 2
특허번호: 1020247038345, 점수: 2.1040, 청구항 수: 2
특허번호: 1020210140662, 점수: 2.1031, 청구항 수: 2
특허번호: 1020207026278, 점수: 2.0657, 청구항 수: 3
특허번호: 1020257031156, 점수: 1.9736, 청구항 수: 2
특허번호: 1020237004752, 점수: 1.8716, 청구항 수: 1
특허번호: 1020227006980, 점수: 1.8567, 청구항 수: 1
특허번호: 1020247031609, 점수: 1.8417, 청구항 수: 1
특허번호: 1020230109044, 점수: 1.8153, 청구항 수: 1
특허번호: 1020217039153, 점수: 1.7968, 청

### 평가

In [7]:
# 정답 데이터 로드
import pandas as pd

data = pd.read_csv('./test_data.csv')
patent_ids = set(data['출원번호'].astype(str))  

# 확인
print(f"총 {len(patent_ids)}개의 출원번호")
print("샘플:", list(patent_ids)[:5])

총 500개의 출원번호
샘플: ['1020220048305', '1020227020666', '1019950044290', '1020210025842', '1020197018643']


In [8]:
# eval 함수 
def eval(multi_query_results, query_list, ipc_list, TOP_K):
    # hybrid_search 호출
    final_response = hybrid_search(multi_query_results, query_list, TOP_K)
    
    # 비교
    response_patent_id = [f['patent_id'] for f in final_response]
    
    print(ipc_list & set(response_patent_id))
    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)
    print("Recall:", len(ipc_list & set(response_patent_id)) / len(ipc_list))
    
    return final_response

In [9]:
# 결과 확인
query = [
    "영상 처리 3D Mapping 시스템 소프트웨어 데이터 처리 MCU 제어부", 
    "컴퓨터 입력 장치 키보드 터치판 사용자 인터페이스 소프트웨어 시스템"
]

results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

TOP_K = 100
eval(results, query, patent_ids, TOP_K)

{'1020257006522'}
Precision: 0.01
Recall: 0.002


[{'patent_id': '1020257003112',
  'score': np.float64(4.403090928598063),
  'top_claim': '그래픽 사용자 인터페이스 시스템',
  'top_claim_no': 1,
  'claims_found': 1,
  'claims': [{'id': '1020257003112_claim1',
    'document': '그래픽 사용자 인터페이스 시스템',
    'distance': 0.3983451724052429,
    'hybrid_score': np.float64(4.881212142886738)}]},
 {'patent_id': '1020247038970',
  'score': np.float64(3.8302791009901407),
  'top_claim': '상기 시스템 사용자 인터페이스는 상기 컴퓨터 시스템에 대한 제어 사용자 인터페이스인, 방법.',
  'top_claim_no': 9,
  'claims_found': 1,
  'claims': [{'id': '1020247038970_claim9',
    'document': '상기 시스템 사용자 인터페이스는 상기 컴퓨터 시스템에 대한 제어 사용자 인터페이스인, 방법.',
    'distance': 0.43581539392471313,
    'hybrid_score': np.float64(4.244754556655712)}]},
 {'patent_id': '1020230170849',
  'score': np.float64(3.767105525357253),
  'top_claim': '상기 제1 및 제2 소프트웨어 인터페이스 각각은 하나 이상의 입력 필드를 표시하도록 구성되고, 상기 하나 이상의 입력 필드는 비-키보드 입력을 수용하도록 구성되는, 컴퓨터-구현 방법.',
  'top_claim_no': 19,
  'claims_found': 2,
  'claims': [{'id': '1020230170849_claim9',
  

---

### 기존 함수와 비교

In [10]:
# eval 함수 - many_claim_dis 사용
def eval(multi_query_results, query_list, ipc_list, TOP_K):
    # many_claim_dis 호출 
    final_response = many_claim_dis(multi_query_results, TOP_K)
    
    # 비교
    response_patent_id = [f['patent_id'] for f in final_response]
    
    print(ipc_list & set(response_patent_id))
    print("Precision:", len(ipc_list & set(response_patent_id)) / TOP_K)
    print("Recall:", len(ipc_list & set(response_patent_id)) / len(ipc_list))
    
    return final_response


In [11]:
# 결과 확인
query = [
    "영상 처리 3D Mapping 시스템 소프트웨어 데이터 처리 MCU 제어부", 
    "컴퓨터 입력 장치 키보드 터치판 사용자 인터페이스 소프트웨어 시스템"
]

# 1. multi_query_rerank 먼저 실행
results = multi_query_rerank(
    collection=collection,
    model=model,
    query_list=query,
    per_query_top_k=200,
    final_top_k=200
)

# 2. eval 함수 호출 (내부에서 many_claim_dis 사용)
TOP_K = 100
final_response = eval(results, query, patent_ids, TOP_K)


set()
Precision: 0.0
Recall: 0.0
